In [4]:
# 이거 사용
import json
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time
from datetime import datetime

# MySQL 연결 정보 설정
config = {
    'user': 'yangjinan',  # 사용자 이름
    'password': 'ALsk85200!!',  # 비밀번호
    'host': 'hanaex.mysql.database.azure.com',  # 호스트
    'database': 'my_database',  # 데이터베이스 이름
    'port': 3306  # 포트 번호
}

# state 변수 (EUR, USD 등)
state = "CNY"

# 오늘 날짜
today_date = datetime.today().strftime('%Y-%m-%d')

# 웹 드라이버 설정 (예: Chrome)
driver = webdriver.Chrome()

# 1. 페이지 로드
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_07i.do#//HanaBank"
driver.get(url)

# 2. 페이지 로드 대기
wait = WebDriverWait(driver, 10)

# 3. 로딩 레이어가 사라질 때까지 대기
try:
    loading_layer = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#OPB_loadingLayerID_generatedByJSOPB_modalMaskID_generatedByJS")))
    wait.until(EC.invisibility_of_element(loading_layer))
except TimeoutException:
    print("Loading layer did not disappear, continuing...")

# 6. 통화 선택 (EUR:유로(유럽연합))
currency_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#curCd")))
select = Select(currency_select)
select.select_by_value(state)

# 7. "조회" 버튼 클릭
query_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
query_button.click()

# 8. 조회 후 5초 대기
time.sleep(5)

# 9. 데이터 추출
data_list = []
rows = driver.find_elements(By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table > tbody > tr")

for row in rows:
    columns = row.find_elements(By.TAG_NAME, "td")
    data = [column.text for column in columns]
    
    # 필요한 값을 float로 변환 (첫 번째 값은 제외)
    for i in range(2, len(data)):
        try:
            data[i] = float(data[i].replace(',', ''))  # 쉼표 제거 후 float 변환
        except ValueError:
            print(f"변환 실패: {data[i]}")
    
    # 데이터 리스트에 추가
    data_list.append(data)
    print("Row:", data)

# 웹 드라이버 종료
driver.quit()

# 10. 데이터베이스에 데이터 저장
try:
    # MySQL 데이터베이스 연결
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    
    # SQL INSERT 문 작성
    query = """
    INSERT INTO t_ex_table 
    (state, period, time, cash_buy, cash_sell, remit_send, remit_receive, 
    foreign_currency_check_sell, exchange_rate, compared_to_previous, exchange_fee_rate, usd_conversion_rate, date) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # 데이터베이스에 저장할 값 준비
    for data in data_list:
        values = (
            state,         # state 변수 저장
            int(data[0]),  # period
            data[1],       # time
            data[2],       # cash_buy
            data[3],       # cash_sell
            data[4],       # remit_send
            data[5],       # remit_receive
            data[6],       # foreign_currency_check_sell
            data[7],       # exchange_rate
            data[8],       # compared_to_previous
            data[9],       # exchange_fee_rate
            data[10],      # usd_conversion_rate
            today_date     # 오늘 날짜 저장
        )
        
        # 데이터 삽입
        cursor.execute(query, values)
    
    # 트랜잭션 커밋
    conn.commit()

except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    # 커넥션 닫기
    cursor.close()
    conn.close()

print("데이터 저장 완료!")

Row: ['224', '12:47:51', 199.72, 180.7, 192.11, 188.31, 0.0, 190.21, 0.0, 4.27712, 0.1413]
Row: ['223', '12:45:46', 199.72, 180.7, 192.11, 188.31, 0.0, 190.21, 0.02, 4.27712, 0.1413]
Row: ['222', '12:44:16', 199.69, 180.69, 192.09, 188.29, 0.0, 190.19, 0.0, 4.27712, 0.1413]
Row: ['221', '12:42:21', 199.69, 180.69, 192.09, 188.29, 0.0, 190.19, 0.01, 4.27712, 0.1413]
Row: ['220', '12:41:51', 199.68, 180.68, 192.08, 188.28, 0.0, 190.18, 0.02, 4.27712, 0.1413]
Row: ['219', '12:41:37', 199.71, 180.69, 192.1, 188.3, 0.0, 190.2, 0.03, 4.27712, 0.1413]
Row: ['218', '12:41:06', 199.67, 180.67, 192.07, 188.27, 0.0, 190.17, 0.04, 4.27712, 0.1413]
Row: ['217', '12:40:36', 199.72, 180.7, 192.11, 188.31, 0.0, 190.21, 0.01, 4.27712, 0.1413]
Row: ['216', '12:39:56', 199.73, 180.71, 192.12, 188.32, 0.0, 190.22, 0.01, 4.27712, 0.1413]
Row: ['215', '12:35:56', 199.74, 180.72, 192.13, 188.33, 0.0, 190.23, 0.01, 4.27712, 0.1412]
Row: ['214', '12:34:22', 199.75, 180.73, 192.14, 188.34, 0.0, 190.24, 0.03, 4.

Row: ['132', '11:21:16', 199.78, 180.76, 192.17, 188.37, 0.0, 190.27, 0.01, 4.27712, 0.1412]
Row: ['131', '11:20:36', 199.79, 180.77, 192.18, 188.38, 0.0, 190.28, 0.02, 4.27712, 0.1412]
Row: ['130', '11:19:57', 199.77, 180.75, 192.16, 188.36, 0.0, 190.26, 0.01, 4.27712, 0.1412]
Row: ['129', '11:19:51', 199.78, 180.76, 192.17, 188.37, 0.0, 190.27, 0.03, 4.27712, 0.1412]
Row: ['128', '11:19:26', 199.81, 180.79, 192.2, 188.4, 0.0, 190.3, 0.02, 4.27712, 0.1412]
Row: ['127', '11:18:28', 199.83, 180.81, 192.22, 188.42, 0.0, 190.32, 0.04, 4.27712, 0.1412]
Row: ['126', '11:18:22', 199.87, 180.85, 192.26, 188.46, 0.0, 190.36, 0.02, 4.27712, 0.1412]
Row: ['125', '11:18:08', 199.89, 180.87, 192.28, 188.48, 0.0, 190.38, 0.05, 4.27712, 0.1412]
Row: ['124', '11:18:01', 199.95, 180.91, 192.33, 188.53, 0.0, 190.43, 0.03, 4.27712, 0.1412]
Row: ['123', '11:17:46', 199.98, 180.94, 192.36, 188.56, 0.0, 190.46, 0.03, 4.27712, 0.1412]
Row: ['122', '11:13:51', 200.01, 180.97, 192.39, 188.59, 0.0, 190.49, 0.0

Row: ['40', '09:37:07', 200.16, 181.1, 192.53, 188.73, 0.0, 190.63, 0.01, 4.27712, 0.1412]
Row: ['39', '09:35:37', 200.17, 181.11, 192.54, 188.74, 0.0, 190.64, 0.05, 4.27712, 0.1412]
Row: ['38', '09:31:22', 200.11, 181.07, 192.49, 188.69, 0.0, 190.59, 0.03, 4.27712, 0.1412]
Row: ['37', '09:31:01', 200.08, 181.04, 192.46, 188.66, 0.0, 190.56, 0.05, 4.27712, 0.1412]
Row: ['36', '09:27:11', 200.14, 181.08, 192.51, 188.71, 0.0, 190.61, 0.08, 4.27712, 0.1412]
Row: ['35', '09:26:57', 200.05, 181.01, 192.43, 188.63, 0.0, 190.53, 0.02, 4.27712, 0.1412]
Row: ['34', '09:23:42', 200.07, 181.03, 192.45, 188.65, 0.0, 190.55, 0.04, 4.27712, 0.1412]
Row: ['33', '09:23:16', 200.03, 180.99, 192.41, 188.61, 0.0, 190.51, 0.05, 4.27712, 0.1412]
Row: ['32', '09:22:56', 200.08, 181.04, 192.46, 188.66, 0.0, 190.56, 0.03, 4.27712, 0.1412]
Row: ['31', '09:22:31', 200.11, 181.07, 192.49, 188.69, 0.0, 190.59, 0.02, 4.27712, 0.1412]
Row: ['30', '09:22:11', 200.14, 181.08, 192.51, 188.71, 0.0, 190.61, 0.02, 4.2771

In [7]:
# 실시간 크롤링
import json
import mysql.connector
from mysql.connector import Error
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, ElementClickInterceptedException,
    NoSuchElementException, StaleElementReferenceException
)
import time
from datetime import datetime

# MySQL 연결 정보 설정
config = {
    'user': 'yangjinan',  # 사용자 이름
    'password': 'ALsk85200!!',  # 비밀번호
    'host': 'hanaex.mysql.database.azure.com',  # 호스트
    'database': 'my_database',  # 데이터베이스 이름
    'port': 3306  # 포트 번호
}

# state 변수 (EUR, USD 등)
state = "USD"

# 오늘 날짜
today_date = datetime.today().strftime('%Y-%m-%d')

# MySQL 데이터베이스 연결
try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
except mysql.connector.Error as err:
    print(f"Database connection error: {err}")
    exit(1)  # 연결 실패 시 프로그램 종료

# 웹 드라이버 설정 (예: Chrome)
driver = webdriver.Chrome()

# URL 설정
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_07i.do#//HanaBank"

try:
    while True:
        # 1. 페이지 로드
        driver.get(url)

        # 2. 페이지 로드 대기
        wait = WebDriverWait(driver, 10)

        # 3. 로딩 레이어가 사라질 때까지 대기
        try:
            loading_layer = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#OPB_loadingLayerID_generatedByJSOPB_modalMaskID_generatedByJS")))
            wait.until(EC.invisibility_of_element(loading_layer))
        except TimeoutException:
            print("Loading layer did not disappear, continuing...")
        except NoSuchElementException:
            pass  # 로딩 레이어가 없으면 통과

        # 6. 통화 선택 (예: EUR)
        try:
            currency_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#curCd")))
            select = Select(currency_select)
            select.select_by_value(state)
        except Exception as e:
            print(f"Error selecting currency: {e}")
            continue  # 다음 루프로 이동

        # 7. "조회" 버튼 클릭
        try:
            query_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
            query_button.click()
        except Exception as e:
            print(f"Error clicking query button: {e}")
            continue

        # 8. 조회 후 5초 대기
        time.sleep(5)

        # 9. 데이터 추출 (첫 번째 행만)
        try:
            first_row = driver.find_element(By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table > tbody > tr")
            columns = first_row.find_elements(By.TAG_NAME, "td")
            data = [column.text for column in columns]

            # 필요한 값을 float로 변환 (첫 번째 두 개의 값은 제외)
            for i in range(2, len(data)):
                try:
                    data[i] = float(data[i].replace(',', ''))  # 쉼표 제거 후 float 변환
                except ValueError:
                    print(f"변환 실패: {data[i]}")

            print("추출된 데이터:", data)
        except Exception as e:
            print(f"Error extracting data: {e}")
            continue

        # 10. 데이터베이스에서 기존 period 조회 및 비교
        try:
            # 동일한 state와 date에 대한 최대 period 조회
            cursor.execute("""
                SELECT MAX(period) FROM t_ex_table WHERE state = %s AND date = %s
            """, (state, today_date))
            result = cursor.fetchone()
            stored_period = result[0] if result[0] is not None else 0
        except mysql.connector.Error as err:
            print(f"Database query error: {err}")
            continue

        # period 비교 및 데이터 삽입
        try:
            scraped_period = int(data[0])
        except ValueError:
            print(f"Invalid period value: {data[0]}")
            continue

        if scraped_period > stored_period:
            # 데이터베이스에 데이터 저장
            try:
                query = """
                INSERT INTO t_ex_table 
                (state, period, time, cash_buy, cash_sell, remit_send, remit_receive, 
                foreign_currency_check_sell, exchange_rate, compared_to_previous, exchange_fee_rate, usd_conversion_rate, date) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                values = (
                    state,         # state 변수 저장
                    scraped_period,  # period
                    data[1],       # time
                    data[2],       # cash_buy
                    data[3],       # cash_sell
                    data[4],       # remit_send
                    data[5],       # remit_receive
                    data[6],       # foreign_currency_check_sell
                    data[7],       # exchange_rate
                    data[8],       # compared_to_previous
                    data[9],       # exchange_fee_rate
                    data[10],      # usd_conversion_rate
                    today_date     # 오늘 날짜 저장
                )
                cursor.execute(query, values)
                conn.commit()
                print("데이터 저장 완료!")
            except mysql.connector.Error as err:
                print(f"Database insertion error: {err}")
        else:
            print("새로 추출한 period가 저장된 period보다 크지 않습니다. 데이터 저장을 생략합니다.")

        # 1분 대기
        time.sleep(5)

except KeyboardInterrupt:
    print("프로세스가 사용자에 의해 중단되었습니다.")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

finally:
    # 리소스 정리
    driver.quit()
    cursor.close()
    conn.close()
    print("리소스를 정상적으로 종료하였습니다.")

추출된 데이터: ['371', '17:05:12', 1372.6, 1325.4, 1362.2, 1335.8, 1333.31, 1349.0, 0.1, 6.65396, 1.0]
데이터 저장 완료!
Loading layer did not disappear, continuing...
프로세스가 사용자에 의해 중단되었습니다.
리소스를 정상적으로 종료하였습니다.
